## 初期設定的なもの

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [306]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dsigmoid(x):
    return x*(1 - x)

class Layer:
    def __init__(self, input_size, output_size):
        self.input_size = input_size                           # 入力数
        self.output_size = output_size                         # ニューロン数と同じ
        self.neurons_W = (np.random.rand(output_size, input_size) - 0.5) * 0.1 # ニューロンごとに入力と同数の足がある分の重み
        self.neurons_b = (np.random.rand(output_size) - 0.5) * 0.2             # ニューロンごとのしきい値
        self.neurons_d = np.zeros(output_size)                                 # ニューロンごとの誤差修正値
    
    def h(self, x):
        return sigmoid(x)
        #return tanh(x)
        #return relu(x)
    
    def dh(self, y):
        return dsigmoid(y)
        #return dtanh(y)
        #return drelu(y)
    
    def output_layer(self, X):
        y = np.zeros(self.output_size)
        for i in range(self.output_size):
            # y = h(sum(W*X) - b)
            y[i] = self.h(np.sum(self.neurons_W[i]*X) - self.neurons_b[i])
        return y

    def train_layer(self, X):
        for i in range(self.output_size):
            # W = W - (-alpha*d*X)
            self.neurons_W[i] -= -alpha*self.neurons_d[i]*X
        return
    
    # 出力層での自分自身の誤差修正値をセット
    def backprop_layer_output(self, O, T):
        # d = (T - O)*dh(O)
        self.neurons_d = (T - O)*self.dh(O)
        return

    # 前のレイヤーへの誤差修正値を算出
    def backprop_layer(self):
        total_prev_D = np.zeros(self.input_size)
        for i in range(self.output_size):
            #total_prev_D += W*d
            total_prev_D += self.neurons_W[i]*self.neurons_d[i]
        return total_prev_D

    # 後のレイヤーで計算された誤差修正値に出力値を掛けて自レイヤーに値をセット
    def set_layer_d(self, D, O):
        # d = D*dh(O)
        self.neurons_d = D*self.dh(O)


In [5]:
def smaller(image):
    def seri(x, y):
        return y*28 + x

    simages = np.zeros(14*14)
    for i in range(14):
        for j in range(14):
            x, y = j*2, i*2
            avg = (image[seri(x+0, y+0)] + image[seri(x+0, y+1)] + image[seri(x+1, y+0)] + image[seri(x+1, y+1)])/4.0
            simages[i*14 + j] = avg
    return simages

In [76]:
plt.figure(figsize=(20,10))

In [103]:
def plotInput(image, w, h, x, y):
    plt.subplot(w, h, y*w + x + 1)
    plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
    plt.tick_params(axis='y', which='both', left='off', right='off', labelleft='off')
    plt.imshow(image, vmin=0, vmax=1, cmap=plt.cm.gray_r, interpolation="nearest")


## numpyで配列の範囲の扱い

In [307]:
test1 = np.array([[11, 12, 13, 14], [21, 22, 23, 24], [31, 32, 33, 34]])

In [311]:
test1[1:3, 0:2]

array([[21, 22],
       [31, 32]])

In [313]:
test2 = test1[1:3, 0:2]

In [314]:
test2[:] = test2 * 2

In [315]:
test1

array([[11, 12, 13, 14],
       [42, 44, 23, 24],
       [62, 64, 33, 34]])

In [317]:
test1.shape

(3, 4)

In [318]:
test1.reshape(-1)

array([11, 12, 13, 14, 42, 44, 23, 24, 62, 64, 33, 34])

In [319]:
test1.reshape(-1).reshape(2, 3, 2)

array([[[11, 12],
        [13, 14],
        [42, 44]],

       [[23, 24],
        [62, 64],
        [33, 34]]])

## convolutionとpoolingの順方向実装

In [302]:
conv1 = Layer(3*3, 20) # 3x3の入力で20種類まで判定出来るたたみこみ層

In [303]:
def conv(conv, inp):
    (h, w) = inp.shape
    # convolution
    (cw, ch) = 3, 3
    (out_w, out_h) = w - (cw - 1), h - (ch - 1)
    out = np.zeros((out_h, out_w, 20))  # 縦12 x 横12 x 種20 の出力結果用
    for y in range(out_h):
        for x in range(out_w):
            window = inp[y:y+ch, x:x+cw].reshape(-1) # 窓を切り出して1次元配列にする
            out[y, x] = conv.output_layer(window)
            #print("y:" + str(y) + " x:" + str(x) + " data:" + str(window))
            #plotInput(inp[y:y+ch, x:x+cw], w, h, x, y)
    return out

def pooling(conv_out):
    (conv_out_h, conv_out_w, conv_out_z) = conv_out.shape
    # pooling 2x2 の範囲で最大のものを抽出
    (pw, ph) = 2, 2
    pooling = np.zeros((conv_out_h / ph, conv_out_w / pw, 20))  # 縦6 x 横6 x 種20 の出力結果用
    for y in range(0, conv_out_h, ph):
        for x in range(0, conv_out_w, pw):
            window = conv_out[y:y+ph, x:x+pw]
            # どの窓のどのパターンが一番反応したかを割り出す
            value = window.max()
            index = window.argmax()
            (window_y, window_x, z) = np.unravel_index(index, window.shape) # 配列が y, x, z 順の構造になってるの注意
            (max_y, max_x) = window_y + y, window_x + x
            pooling[y / ph, x / pw, z] = value # そのpoolingで最大のものを詰める
            #print("y:" + str(max_y) + " x:" + str(max_x) + " z:" + str(z) + " value:" + str(value))
    return pooling

def train_pooling(conv, inp, conv_out):
    (conv_out_h, conv_out_w, conv_out_z) = conv_out.shape
    # convolution
    (cw, ch) = 3, 3
    # pooling 2x2 の範囲で最大のものを抽出
    (pw, ph) = 2, 2
    for y in range(0, conv_out_h, ph):
        for x in range(0, conv_out_w, pw):
            window = conv_out[y:y+ph, x:x+pw]
            # どの窓のどのパターンが一番反応したかを割り出す
            value = window.max()
            index = window.argmax()
            (window_y, window_x, z) = np.unravel_index(index, window.shape) # 配列が y, x, z 順の構造になってるの注意
            (max_y, max_x) = window_y + y, window_x + x
            #print("y:" + str(max_y) + " x:" + str(max_x) + " z:" + str(z) + " value:" + str(value))
            # 最大のやつだけを「1」を出すよう訓練
            conv_inp = inp[max_y:max_y+ch, max_x:max_x+cw].reshape(-1)
            print("y:" + str(max_y) + " x:" + str(max_x) + " z:" + str(z) + " value:" + str(value))
            #conv.train_neuron(conv.w[z], conv.b[z], conv_inp, 1)
            t = np.zeros(20)
            t[z] = 1.0
            conv.train_layer(conv_inp, t)


In [304]:
alpha = 0.1

fig = plt.figure(figsize=(16,8))

def train_neocognitron(train_count):
    images, labels = mnist.train.next_batch(train_count)
    square_error = 0.0
    for i in range(len(images)):
        X = smaller(images[i]).reshape(14, 14)
        Y = conv(conv1, X)
        #print(pooling(Y))
        train_pooling(conv1, X, Y)


In [ ]:
train_neocognitron(100)